# Step 1: Loading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('credits.csv')
df.head()

,person_id,id,name,character,role
0,14701,tm77588,Humphrey Bogart,Rick Blaine,ACTOR
1,14702,tm77588,Ingrid Bergman,Ilsa Lund,ACTOR
2,14703,tm77588,Paul Henreid,Victor Laszlo,ACTOR
3,14704,tm77588,Claude Rains,Captain Louis Renault,ACTOR
4,14705,tm77588,Conrad Veidt,Major Heinrich Strasser,ACTOR


# Step 2: Shuffle and Split the Data

In [13]:
# Shuffle the data
df_shuffled = df.sample(frac=1, random_state=42)

# Calculate the number of rows for each destination
total_rows = df_shuffled.shape[0]
json_rows = int(0.3 * total_rows)
database_rows = int(0.3 * total_rows)
csv_rows = total_rows - json_rows - database_rows

# Split the data into JSON, database, and CSV parts
df_json = df_shuffled.iloc[:json_rows]
df_database = df_shuffled.iloc[json_rows:json_rows+database_rows]
df_csv = df_shuffled.iloc[json_rows+database_rows:]

In [14]:
df_database.head()

,person_id,id,name,character,role
3905,155030,tm84135,Billy Wells,Minor Role (uncredited),ACTOR
99,14793,tm77588,Leo White,Emile - Waiter (uncredited),ACTOR
59036,1249260,tm956831,Ryan Dillon,Elmo,ACTOR
31546,24461,tm79639,Ramon Frank,Kinter,ACTOR
62526,1954864,tm463117,Lizzy Plimpton,Softball Parent #2,ACTOR


# Save the JSON and CSV Data

In [15]:
# Save the splits to their respective destinations (JSON, database, and CSV)
df_json.to_json('data.json', orient='records', lines=True)
df_csv.to_csv('data.csv', index=False)

# Step 5: Transfer data to the database

In [16]:
import pyodbc

In [18]:
# Créez la connexion
connection = pyodbc.connect('DRIVER={SQL SERVER} ;SERVER=LAPTOP-1US3GU3J\SQLEXPRESS;DATABASE=DBpart')

# Create a cursor to execute SQL queries
cursor = connection.cursor()

In [ ]:
cursor.execute('''
    CREATE TABLE partdb (
        person_id INT,
        id NVARCHAR(255),
        name NVARCHAR(255),
        character NVARCHAR(255),
        role NVARCHAR(255),
        PRIMARY KEY (person_id)
    );
''')

In [ ]:
# Insert data into the dbpart table
for row in df_database.itertuples():
    # Check if the person_id already exists in the table
    cursor.execute('SELECT COUNT(*) FROM partdb WHERE person_id = ?', (row.person_id,))
    count = cursor.fetchone()[0]
    
    if count == 0:
        # Insert the record if it doesn't exist
        cursor.execute('INSERT INTO partdb (person_id, id, name, character, role) VALUES (?, ?, ?, ?, ?)', (row.person_id, row.id, row.name, row.character, row.role))

connection.commit()

In [ ]:
# Close the connection
cursor.close()
connection.close()